### Our Recommendation System
We will be design a more robust recommendation process, carried out with graph data.

Recommendation systems are used heavily in customer-facing applications, to show the user a product that they may be interested in. Product recommendations are often based on what behaviorally similar users have played and purchased.

##### Generic MySQL to igraph methods
We will write a set of reusable, geneeric methods to create an igraph graph from columns in a MySQL table. The functions will be designed to create a heterogeneous, bipartile, directed graph, give a set of column names.

In [2]:
import igraph as ig
from graphtastic.database.mysql import query_mysql
from graphtastic.igraph.helper import create_igraph_ids

In [1]:
def mysql_to_graph(table, source, target, weights, password):
    
    sql_query = f"SELECT {source}, {target}, {weights} FROM {table};"
    
    data = query_mysql(sql_query, password = password)
    
    source_nodes = sorted(list(set([source for source, _, _ in data])))
    
    target_nodes = sorted(list(set([target for _, target, _ in data])))
    
    # Create a dictionaries using list comprehension
    source_igraph_ids = create_igraph_ids(source_nodes)
    target_igraph_ids = create_igraph_ids(target_nodes, len(source_igraph_ids))
    
    # we need to generate an igraph edge list using the pairs in the created dictionaries
    edges = [(source_igraph_ids[source], target_igraph_ids[target]) for source, target, _ in data]
    weights = [weight for _, _, weight in data]
    
    # The data is ready to be added to an igraph graph.
    g = ig.Graph(directed=True)
    g.add_vertices(len(source_nodes + target_nodes))
    g.vs['internal_id'] = list(source_igraph_ids.keys()) + list(target_igraph_ids.keys())
    g.vs['type'] = ['source' for _ in source_nodes] + ['target' for _ in target_nodes]
    g.add_edges(edges)
    g.es['weight'] = weights
    
    return g

In [4]:
g = mysql_to_graph('steam_play', 'id', 'game_name', 'hours', password='')